# Spark

We will perform basic data processing tasks within Spark using the concept of Resilient Distributed Datasets (RDDs).

In [1]:
import pyspark
from pyspark import SparkConf, SparkContext

We run Spark in [local mode](http://spark.apache.org/docs/latest/programming-guide.html#local-vs-cluster-modes) from within our Docker container.

In [2]:
sc = SparkContext('local[*]')

We create a new RDD by reading in the data as a text file. We use the ratings data from [MovieLens](http://grouplens.org/datasets/movielens/latest/).

In [3]:
text_file = sc.textFile('/home/data_scientist/data/ml-latest-small/ratings.csv')

- Function `read_ratings_csv` creates a new RDD by transforming `text_file` into an RDD with columns of appropriate data types.
- The function accepts a `pyspark.rdd.RDD` instance (e.g., `text_file` in the above code cell) and returns another RDD instance, `pyspark.rdd.PipelinedRDD`.
- `ratings.csv` contains a header row. Use the `head` command or otherwise to inspect the file.

In [4]:
def read_ratings_csv(rdd):
    '''
    Creates an RDD by transforming `ratings.csv`
    into columns with appropriate data types.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.rdd.PipelinedRDD instance.
    '''
    
    col_data = rdd.map(lambda l: l.split(",")) \
            .map(lambda p: (p[0], p[1], p[2], p[3])) \
            .filter(lambda line: 'movieId' not in line)
            
    cols = col_data.filter(lambda line: 'NA' not in line)
    
    fields = cols.map(lambda p: (int(p[0]), int(p[1]), float(p[2]), int(p[3])))
    
    return fields

In [5]:
ratings = read_ratings_csv(text_file)
print(ratings.take(3))

[(1, 16, 4.0, 1217897793), (1, 24, 1.5, 1217895807), (1, 32, 4.0, 1217896246)]


For simplicity, we might want to restrict our analysis to only favorable ratings, which, since the movies are rated on a five-star system, we take to mean ratings greater than three. So

- Function `filter_favorable_ratings` selects rows whose rating is greater than 3.

In [7]:
def filter_favorable_ratings(rdd):
    '''
    Selects rows whose rating is greater than 3.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.rdd.PipelinedRDD instance.
    '''
    
    fav = rdd.filter(lambda p: p[2]>3)
    
    return fav

In [8]:
favorable = filter_favorable_ratings(ratings)

We might also want to select only those movies that have been reviewed by multiple people.

- Function `find_n_reviews` that returns the number of reviews for a given movie.

In [10]:
def find_n_reviews(rdd, movie_id):
    '''
    Finds the number of reviews for a movie.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    movie_id: An int.
    
    Returns
    -------
    A pyspark.rdd.PipelinedRDD instance.
    '''
    
    counts = rdd.filter(lambda p: p[1]==movie_id)
    
    return counts.count()

In [11]:
n_toy_story = find_n_reviews(favorable, 1)
print(n_toy_story)

172


## Cleanup

We must stop the SparkContext in order to release the spark resources before existing this Notebook.

In [13]:
sc.stop()

# Spark DataFrames

In this problem, we will use the Spark DataFrame to perform basic data processing tasks.

In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
import pandas as pd

We run Spark in [local mode](http://spark.apache.org/docs/latest/programming-guide.html#local-vs-cluster-modes) from within our Docker container.

In [2]:
sc = SparkContext('local[*]')

We create a new RDD by reading in the data as a textfile. We use the ratings data from [MovieLens](http://grouplens.org/datasets/movielens/latest/). See [Week 6 Lesson 1](https://github.com/UI-DataScience/info490-sp16/blob/master/Week6/notebooks/intro2rs.ipynb) for more information on this data set.

In [3]:
csv_path = '/home/data_scientist/data/ml-latest-small/ratings.csv'
text_file = sc.textFile(csv_path)

- Function `create_df` creates a Spark DataFrame from `text_file`. 

In [4]:
def create_df(rdd):
    '''
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.sql.dataframe.DataFrame instance.
    '''
    
    col_data = rdd.map(lambda l: l.split(",")) \
            .map(lambda p: (p[0], p[1], p[2], p[3])) \
            .filter(lambda line: 'movieId' not in line)
            
    cols = col_data.filter(lambda line: 'NA' not in line)
    
    fields = cols.map(lambda p: (int(p[0]), int(p[1]), float(p[2]), int(p[3])))
    
    sqlContext = SQLContext(sc)

    schemaString = "userId movieId rating timestamp"
    
    fieldTypes = [IntegerType(), IntegerType(), FloatType(), IntegerType()]
    
    f_data = [StructField(field_name, field_type, True) \
          for field_name, field_type in zip(schemaString.split(), fieldTypes)]
    
    schema = StructType(f_data)
    
    df = sqlContext.createDataFrame(fields, schema)
    
    return df

In [5]:
df = create_df(text_file)
df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     16|   4.0|1217897793|
|     1|     24|   1.5|1217895807|
|     1|     32|   4.0|1217896246|
|     1|     47|   4.0|1217896556|
|     1|     50|   4.0|1217896523|
|     1|    110|   4.0|1217896150|
|     1|    150|   3.0|1217895940|
|     1|    161|   4.0|1217897864|
|     1|    165|   3.0|1217897135|
|     1|    204|   0.5|1217895786|
|     1|    223|   4.0|1217897795|
|     1|    256|   0.5|1217895764|
|     1|    260|   4.5|1217895864|
|     1|    261|   1.5|1217895750|
|     1|    277|   0.5|1217895772|
|     1|    296|   4.0|1217896125|
|     1|    318|   4.0|1217895860|
|     1|    349|   4.5|1217897058|
|     1|    356|   3.0|1217896231|
|     1|    377|   2.5|1217896373|
+------+-------+------+----------+
only showing top 20 rows



- Select from the Spark DataFrame only the rows whose rating is greater than 3.
- After filtering, return only two columns: `movieId` and `rating`.

In [7]:
def filter_favorable_ratings(df):
    '''
    Selects rows whose rating is greater than 3.
    
    Parameters
    ----------
    A pyspark.sql.dataframe.DataFrame instance.

    Returns
    -------
    A pyspark.sql.dataframe.DataFrame instance.

    '''
    
    df1 = df.filter(df['rating'] >3)
    
    return df1['movieId','rating']

In [8]:
favorable = filter_favorable_ratings(df)
favorable.show()

+-------+------+
|movieId|rating|
+-------+------+
|     16|   4.0|
|     32|   4.0|
|     47|   4.0|
|     50|   4.0|
|    110|   4.0|
|    161|   4.0|
|    223|   4.0|
|    260|   4.5|
|    296|   4.0|
|    318|   4.0|
|    349|   4.5|
|    457|   4.0|
|    480|   3.5|
|    527|   4.5|
|    589|   3.5|
|    590|   3.5|
|    593|   5.0|
|    608|   3.5|
|    648|   3.5|
|    724|   3.5|
+-------+------+
only showing top 20 rows



- Function `find_n_reviews`, given a `movieId`, computes the number of reviews for that movie.

In [10]:
def find_n_reviews(df, movie_id):
    '''
    Finds the number of reviews for a movie.
    
    Parameters
    ----------
    movie_id: An int.
    
    Returns
    -------
    '''
    
    n_reviews = df.filter(df['movieId'] == movie_id).count()
    
    return n_reviews

In [11]:
n_toy_story = find_n_reviews(favorable, 1)
print(n_toy_story)

172


## Cleanup

We must stop the SparkContext in order to release the spark resources before existing this Notebook.

In [13]:
sc.stop()

# Spark MLlib

In this problem, we will use Spark MLlib to perform a logistic regression on the flight data to determine whether a flight would be delayed or not.

In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD

We run Spark in [local mode](http://spark.apache.org/docs/latest/programming-guide.html#local-vs-cluster-modes) from within our Docker container.

In [2]:
sc = SparkContext('local[*]')

We use code similar to the RDD code from the [Introduction to Spark](https://github.com/UI-DataScience/info490-sp16/blob/master/Week14/notebooks/intro2spark.ipynb) notebook to import two columns: `ArrDealy` and `DepDelay`.

In [3]:
text_file = sc.textFile('/home/data_scientist/data/2001.csv')

data = (
    text_file
    .map(lambda line: line.split(","))
    # 14: ArrDelay, 15: DepDelay
    .map(lambda p: (p[14], p[15]))
    .filter(lambda line: 'ArrDelay' not in line)
    .filter(lambda line: 'NA' not in line)
    .map(lambda p: (int(p[0]), int(p[1])))
    )

len_data = data.count()

- Function `to_binary` transforms the `ArrDelay` column into binary labels that indicate whether a flight arrived late or not. We define a flight to be delayed if its arrival delay is 15 minutes or more, the same definition used by the FAA (source: [Wikipedia](https://en.wikipedia.org/wiki/Flight_cancellation_and_delay)).

- The `DepDelay` column should remain unchanged.

In [4]:
def to_binary(rdd):
    '''
    Transforms the "ArrDelay" column into binary labels
    that indicate whether a flight arrived late or not.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.rdd.PipelinedRDD instance.
    '''
    
    def bin_trans(x):
        if x >= 15:
            return 1
        else:
            return 0
    
    rdd = rdd.map(lambda p: (bin_trans(p[0]), p[1]))
    
    return rdd

In [5]:
binary_labels = to_binary(data)
print(binary_labels.take(5))

[(0, -4), (0, -5), (1, 11), (0, -3), (1, 0)]


Our data must be in a Spark specific data structure called [LabeledPoint](https://spark.apache.org/docs/latest/mllib-data-types.html#labeled-point). So

- Function `to_labeled_point` turns a Spark sequence of tuples into a sequence containing LabeledPoint values for each row. The arrival delay should be the label, and the departure delay should be the feature.

In [7]:
def to_labeled_point(rdd):
    '''
    Transforms a Spark sequence of tuples into
    a sequence containing LabeledPoint values for each row.
    
    The arrival delay is the label.
    The departure delay is the feature.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A pyspark.rdd.PipelinedRDD instance.
    '''
    
    rdd = rdd.map(lambda p: LabeledPoint(p[0],[p[1]]))
    
    return rdd

In [8]:
labeled_point = to_labeled_point(binary_labels)
print(labeled_point.take(5))

[LabeledPoint(0.0, [-4.0]), LabeledPoint(0.0, [-5.0]), LabeledPoint(1.0, [11.0]), LabeledPoint(0.0, [-3.0]), LabeledPoint(1.0, [0.0])]


- Use [LogisticRegressionWithSGD](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithSGD) to train a [logistic regression](http://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression) model. 
- Use 10 iterations. Use default parameters for all other parameters other than `iterations`.
- Use the resulting logistic regression model to make predictions on the entire data, and return an RDD of (label, prediction) pairs.

In [10]:
def fit_and_predict(rdd):
    '''
    Fits a logistic regression model.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    An RDD of (label, prediction) pairs.
    '''
    
    log_model = LogisticRegressionWithSGD.train(rdd, iterations=10)
    
    rdd = rdd.map(lambda lp: (lp.label, float(log_model.predict(lp.features))))
    
    return rdd

In [11]:
labels_and_preds = fit_and_predict(labeled_point)
print(labels_and_preds.take(5))

[(0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (1.0, 0.0)]


- Write a function that computes the accuracy from a Spark sequence of (label, prediction) pairs.

In [13]:
def get_accuracy(rdd):
    '''
    Computes accuracy.
    
    Parameters
    ----------
    rdd: A pyspark.rdd.RDD instance.
    
    Returns
    -------
    A float.
    '''
    
    acc = rdd.map(lambda p: p[0]==p[1])
    
    accuracy = acc.mean()
    
    return accuracy

In [14]:
accuracy = get_accuracy(labels_and_preds)
print(accuracy)

0.750394021461391


## Cleanup

We must stop the SparkContext in order to release the spark resources before existing this Notebook.

In [16]:
sc.stop()